In [9]:
# UNCOMMENT THESE TO INSTALL THE REQUIRED LIBRARIES

# %pip install pandas
# %pip install tensorflow
# %pip install scikit-learn
# %pip install pymysql
# %pip install sshtunnel

In [10]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import pymysql
import sshtunnel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Sklearn version:", sklearn.__version__)
print("Pymysql version:", pymysql.__version__)
print("Sshtunnel version:", sshtunnel.__version__)

Numpy version: 1.26.4
Pandas version: 2.2.2
Tensorflow version: 2.16.1
Sklearn version: 1.4.2
Pymysql version: 1.4.6
Sshtunnel version: 0.4.0


### Reading data from a SQL-SERVER

In [11]:
# Set up SSH tunnel
server = sshtunnel.SSHTunnelForwarder(
    ('selene.hud.ac.uk', 22),
    ssh_username='workit',
    ssh_password='umbra(despair>Quartz218',
    remote_bind_address=('localhost', 3306)
)
server.start()

# Defining the connection
connection = pymysql.connect(host='localhost', port=server.local_bind_port, user='workit', password='umbra(despair>Quartz218', database='workit')

In [12]:
MODEL_PATH = "ann_gym_recommender.keras"

# Load the dataset
dataset = pd.read_sql_query("SELECT * FROM `megaGymDataset`", connection)
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


In [13]:
server.stop()

In [14]:
def labelEncodersScaler(data):
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return label_encoders, scaler

In [15]:
def train_model(data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Build the ANN model using TensorFlow
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(data["ID"].unique()), activation='softmax')  # Use the unique number of IDs as the output dimension
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    return model

In [16]:
def recommend_exercises(user_input, model, data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

### Training from dataset

In [17]:
model = train_model(dataset.copy())
model.save("ann_gym_recommender.keras")

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0021 - loss: 7.9912 - val_accuracy: 0.0000e+00 - val_loss: 8.0201
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0040 - loss: 7.8947 - val_accuracy: 0.0000e+00 - val_loss: 8.6870
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0039 - loss: 6.9641 - val_accuracy: 0.0000e+00 - val_loss: 10.3749
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0128 - loss: 6.1600 - val_accuracy: 0.0000e+00 - val_loss: 12.6521
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0169 - loss: 5.6452 - val_accuracy: 0.0000e+00 - val_loss: 14.2470
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0236 - loss: 5.2749 - val_accuracy: 0.0000e+00 - val_loss: 15.7220
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0321 - loss: 5.0029 - val_accuracy: 0.0000e+00 - val_loss: 17.0547
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0307 - loss: 4.8479 -

### Predict from user input

In [18]:
model = load_model(MODEL_PATH)
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input, model, dataset.copy())
dataset.iloc[IDs]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
857,857,UN Pull-Up Neutral,"Pull-up variation with a neutral grip, targeti...",Strength,Biceps,Body Only,Intermediate,9.0
855,855,UN Pull-Up Supinated,Pull-up variation with a supinated (underhand)...,Strength,Biceps,Body Only,Intermediate,9.0
854,854,Pull-up - Gethin Variation,The pull-up is a multi-joint bodyweight exerci...,Strength,Biceps,Body Only,Intermediate,9.0
853,853,TBS Chin-Up,The a chin-up is a variation of the pull-up ex...,Strength,Biceps,Body Only,Intermediate,9.0
909,909,Standing Calf Raise,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833333333333
910,910,Standing calf raise-,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833333333333
684,684,Side-To-Side Adductor Stretch,Stretching the adductor muscles by moving legs...,Strength,Adductors,Body Only,Intermediate,4.816666666666667
694,694,HM Left Leg Swing,Dynamic leg swing exercise targeting hip flexo...,Strength,Adductors,Body Only,Intermediate,4.816666666666667
695,695,UNS Foam Roll Adductor,Using a foam roller to massage and release ten...,Strength,Adductors,Body Only,Intermediate,4.816666666666667
697,697,Side-lying leg adduction scissor,The side-lying leg adduction scissor is a dyna...,Strength,Adductors,Body Only,Intermediate,4.816666666666666
